In [ ]:
# ==========================================================
# DATASET FINAL INCENDIOS + CLIMA
# TARGET DE RIESGO OPERACIONAL (FRP)
# ==========================================================

import pandas as pd
import numpy as np

# ----------------------------------------------------------
# 1. CARGA DEL DATASET BASE
# ----------------------------------------------------------
# Descripción:
# En este paso se importa el dataset consolidado que integra información de focos de incendio detectados por sensores satelitales junto con variables climáticas asociadas
# Esta carga constituye la base empírica del análisis y define el universo de observaciones sobre el cual se aplicarán las transformaciones posteriores.
# ----------------------------------------------------------

df = pd.read_csv(
    "/content/sample_data/Clima_Incendios_Detallado_Clasificado.csv",
    low_memory=False
)

print("Dataset cargado:", df.shape)

# ----------------------------------------------------------
# 2. CONSTRUCCIÓN DE VARIABLE TEMPORAL DEL FOCO
# ----------------------------------------------------------
# Descripción:
# Se construye una variable temporal continua de tipo fecha-hora a partir de la fecha y hora de adquisición del foco
# Este proceso asegura una representación temporal precisa y homogénea, fundamental para el análisis cronológico y la posterior agrupación de focos en ventanas temporales definidas.
# ----------------------------------------------------------

df["acq_date"] = pd.to_datetime(df["acq_date"], errors="coerce")
df["acq_time"] = df["acq_time"].astype(str).str.zfill(4)

df["fecha_inicio"] = pd.to_datetime(
    df["acq_date"].astype(str) + " " +
    df["acq_time"].str[:2] + ":" +
    df["acq_time"].str[2:],
    errors="coerce"
)

# ----------------------------------------------------------
# 3. LIMPIEZA Y DEPURACIÓN DE DATOS
# ----------------------------------------------------------
# Descripción:
# Se realiza una depuración del dataset eliminando registros que presentan valores nulos o inválidos en variables consideradas críticas para el análisis, tales como intensidad del foco (FRP), variables climáticas y localización
# Este paso garantiza un nivel mínimo de calidad y confiabilidad de los datos.
# ----------------------------------------------------------

df = df.dropna(subset=[
    "fecha_inicio",
    "frp",
    "comuna",
    "temperature_2m_mean",
    "relativehumidity_2m_mean",
    "windspeed_10m_mean",
    "precipitation_sum"
]).reset_index(drop=True)

# ----------------------------------------------------------
# 4. DEFINICIÓN DE EVENTOS OPERATIVOS DE INCENDIO
# ----------------------------------------------------------
# Descripción:
# Se operacionaliza el concepto de evento de incendio mediante la combinación de criterios espaciales (comuna mas cercana) y temporales (ventana de 24 horas)
# De esta forma, múltiples focos cercanos en espacio y tiempo son considerados parte de un mismo evento operativo, alineando el análisis con una lógica de gestión y respuesta.
# ----------------------------------------------------------

df["fecha_dia"] = df["fecha_inicio"].dt.floor("D")

df["event_id_final"] = (
    df["comuna"].astype(str) + "_" +
    df["fecha_dia"].astype(str)
)

# ----------------------------------------------------------
# 5. AGREGACIÓN DE INFORMACIÓN A NIVEL EVENTO
# ----------------------------------------------------------
# Descripción:
# Los focos individuales son agrupados según su identificador de evento, permitiendo resumir la información a nivel de evento
# En esta etapa se integran estadísticas espaciales, temporales y de intensidad, reduciendo la granularidad del dataset.
# ----------------------------------------------------------

eventos = (
    df.groupby("event_id_final")
      .agg(
          comuna=("comuna", "first"),
          fecha_dia=("fecha_dia", "first"),
          latitud=("latitud", "mean"),
          longitud=("longitud", "mean"),
          time_start=("fecha_inicio", "min"),
          time_end=("fecha_inicio", "max"),
          frp_inicial=("frp", "min"),
          frp_max=("frp", "max"),
          temperature_2m_mean=("temperature_2m_mean", "mean"),
          relativehumidity_2m_mean=("relativehumidity_2m_mean", "mean"),
          windspeed_10m_mean=("windspeed_10m_mean", "mean"),
          precipitation_sum=("precipitation_sum", "sum")
      )
      .reset_index()
)

# ----------------------------------------------------------
# 6. CÁLCULO DE MÉTRICAS DE INTENSIDAD DEL EVENTO
# ----------------------------------------------------------
# Descripción:
# Se calculan métricas representativas de la intensidad del evento de incendio, utilizando el Fire Radiative Power (FRP) como indicador principal
# Estas métricas permiten caracterizar la severidad del evento desde una perspectiva operacional.
# ----------------------------------------------------------

eventos["duracion_horas"] = (
    (eventos["time_end"] - eventos["time_start"])
    .dt.total_seconds() / 3600
)

eventos = eventos[eventos["duracion_horas"] > 0].reset_index(drop=True)

# ----------------------------------------------------------
# 7. INTEGRACIÓN Y RESUMEN DE VARIABLES CLIMÁTICAS
# ----------------------------------------------------------
# Descripción:
# Se integran y resumen las variables climáticas asociadas a los focos que componen cada evento
# Este paso permite caracterizar el contexto ambiental bajo el cual se desarrolla el incendio, aspecto clave para análisis explicativos y predictivos.
# ----------------------------------------------------------

def clasificar_intensidad(frp):
    if frp < 100:
        return "bajo"
    elif frp < 500:
        return "medio"
    elif frp < 2000:
        return "alto"
    else:
        return "extremo"

eventos["intensidad_incendio"] = eventos["frp_max"].apply(clasificar_intensidad)

# ----------------------------------------------------------
# 8. CLASIFICACIÓN DEL NIVEL DE RIESGO OPERACIONAL
# ----------------------------------------------------------
# Descripción:
# A partir de las métricas de intensidad calculadas, se clasifica cada evento en un nivel de riesgo operacional utilizando umbrales previamente definidos
# Esta clasificación traduce variables continuas en categorías interpretables para la toma de decisiones.
# ----------------------------------------------------------

eventos["target_riesgo"] = (eventos["frp_max"] >= 100).astype(int)

# ----------------------------------------------------------
# 9. VALIDACIÓN Y CONSISTENCIA DEL DATASET
# ----------------------------------------------------------
# Descripción:
# Se realizan verificaciones finales de coherencia y consistencia sobre el dataset agregado, asegurando que las variables calculadas y clasificadas cumplan con los criterios metodológicos definidos para el estudio.
# ----------------------------------------------------------

eventos = eventos.sort_values(["comuna", "fecha_dia"])

lag_vars = [
    "temperature_2m_mean",
    "relativehumidity_2m_mean",
    "windspeed_10m_mean",
    "precipitation_sum"
]

for var in lag_vars:
    for lag in [1, 2, 3]:
        eventos[f"{var}_lag{lag}"] = (
            eventos.groupby("comuna")[var].shift(lag)
        )

# ----------------------------------------------------------
# 10. PREPARACIÓN DEL DATASET FINAL DE ANÁLISIS
# ----------------------------------------------------------
# Descripción:
# Se seleccionan, ordenan y estructuran las variables finales que conformarán el dataset de salida
# Este paso deja el conjunto de datos listo para su utilización en etapas de análisis exploratorio, modelamiento o visualización.
# ----------------------------------------------------------

lag_cols = [c for c in eventos.columns if "_lag" in c]

for col in lag_cols:
    base_col = col.replace("_lag1", "").replace("_lag2", "").replace("_lag3", "")
    eventos[col] = eventos[col].fillna(eventos[base_col])

# ----------------------------------------------------------
# 11. EXPORTACIÓN DEL DATASET FINAL
# ----------------------------------------------------------
# Descripción:
# Finalmente, el dataset resultante es exportado a un archivo, permitiendo su reutilización en etapas posteriores del proyecto o su integración con otros procesos analíticos.
# ----------------------------------------------------------

print("\nEventos finales:", len(eventos))

print("\nDistribución de intensidad:")
print(eventos["intensidad_incendio"].value_counts())

print("\nDistribución target_riesgo:")
print(eventos["target_riesgo"].value_counts(normalize=True))

# ----------------------------------------------------------
# 12. GUARDAR DATASET FINAL
# ----------------------------------------------------------
eventos.to_csv(
    "/content/Dataset_Incendios_Eventos_TARGET_RIESGO_FRP.csv",
    index=False,
    float_format="%.6f"
)

print("\nDataset final guardado correctamente:")
print("Dataset_Incendios_Eventos_TARGET_RIESGO_FRP.csv")


Dataset cargado: (187891, 33)


/tmp/ipython-input-1891280104.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["fecha_inicio"] = pd.to_datetime(



Eventos finales: 2759

Distribución de intensidad:
intensidad_incendio
bajo     2420
medio     327
alto       12
Name: count, dtype: int64

Distribución target_riesgo:
target_riesgo
0    0.877129
1    0.122871
Name: proportion, dtype: float64

Dataset final guardado correctamente:
Dataset_Incendios_Eventos_TARGET_RIESGO_FRP.csv


##Resumen de resultados obtenidos

El proceso de construcción y agregación de eventos de incendio permitió transformar un dataset inicial de 187.891 registros individuales, correspondientes a focos detectados a nivel satelital y variables climáticas asociadas, en un conjunto de 2.759 eventos de incendio definidos operativamente a partir de criterios espaciales (comuna) y temporales (ventana de 24 horas).

Este proceso de agregación reduce significativamente la granularidad de los datos, permitiendo un análisis a nivel de evento, más coherente con una lógica de gestión y evaluación del riesgo operacional.

Distribución de la intensidad del incendio

La clasificación de los eventos según su intensidad, basada en métricas derivadas del Fire Radiative Power (FRP), muestra una distribución marcadamente asimétrica:

Baja intensidad: 2.420 eventos

Intensidad media: 327 eventos

Alta intensidad: 12 eventos

Estos resultados evidencian que la gran mayoría de los eventos corresponden a incendios de baja intensidad, mientras que los eventos de alta intensidad representan una fracción muy reducida del total. Este comportamiento es consistente con la literatura, donde los incendios severos suelen ser menos frecuentes pero de alto impacto operativo.

Distribución del target de riesgo operacional

A partir de la clasificación de intensidad, se definió un target binario de riesgo operacional, donde:

Target 0 (riesgo no crítico): 87,7 % de los eventos

Target 1 (riesgo crítico): 12,3 % de los eventos

Esta distribución confirma la presencia de un desbalance de clases, con predominancia de eventos de bajo riesgo frente a un subconjunto menor de eventos que presentan condiciones de riesgo elevado. Este aspecto resulta especialmente relevante para etapas posteriores de modelamiento predictivo, ya que requerirá estrategias específicas de tratamiento del desbalance para evitar sesgos en el aprendizaje del modelo.